In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
driver = webdriver.Chrome()

url = "https://www.gist.ac.kr/kr/html/sub05/050602.html?mode=V&no=215761"
driver.get(url)

driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="a7f5dbbb988f22a3c0e299be7af8875d")>

In [3]:
iframe = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe"))  # 혹은 By.XPATH, By.ID 등
)
driver.switch_to.frame(iframe)

driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="a7f5dbbb988f22a3c0e299be7af8875d")>

In [4]:
textLayer = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '#pageContainer1 > div.textLayer'))
)

textTags = textLayer.find_elements(By.TAG_NAME, 'div')

[t.text for t in textTags]

['11월 11일 (월)',
 '11월 12일 (화)',
 '11월 13일 (수)',
 '11월 14일 (목)',
 '11월 15일 (금)',
 '11월 16일 (토)',
 '11월 17일 (일)',
 '쌀밥 & ',
 '햄볶음밥 10.15',
 '쌀밥 & ',
 '카레볶음밥 10.15',
 '쌀밥 & ',
 '굴소스볶음밥 10.15',
 '쌀밥 & ',
 '김치볶음밥 10.15',
 '쌀밥 & ',
 '짜장볶음밥 10.15',
 '흑미밥',
 '흑미밥',
 '어묵무국',
 '소고기미역국 16',
 '얼큰콩나물국',
 '건새우미역국 9',
 '시금치된장국',
 '콩나물김치국',
 '계란파국 1',
 '계란후라이 1',
 '계란후라이 1',
 '계란후라이 1',
 '계란후라이 1',
 '계란후라이 1',
 '계란후라이 1',
 '계란후라이 1',
 '깻잎닭갈비 15',
 '돈육메추리알장조림 1.10',
 '팝콘치킨 & 머스타드 15',
 '오징어야채볶음 17',
 '제육버섯볶음 10',
 '새우가스 & 타르S 9',
 '참치야채볶음',
 '청경채오리엔탈무침',
 '고구마순볶음',
 '고추지무침',
 '얼갈이생채',
 '두부조림 5',
 '감자채볶음',
 '깻잎지무침',
 '배추김치',
 '배추김치',
 '배추김치',
 '깍두기',
 '배추김치',
 '배추김치',
 '배추김치',
 '야채샐러드',
 '야채샐러드',
 '야채샐러드',
 '야채샐러드',
 '야채샐러드',
 '야채샐러드',
 '야채샐러드',
 '시리얼&우유 2.6',
 '시리얼&요거트 2.6',
 '시리얼&우유 2.6',
 '시리얼&요거트 2.6',
 '시리얼&우유 2.6',
 '시리얼&우유 2.6',
 '시리얼&우유 2.6',
 '토스트&잼 6',
 '토스트&잼 6',
 '토스트&잼 6',
 '토스트&잼 6',
 '토스트&잼 6',
 '토스트&잼 6',
 '토스트&잼 6',
 '함박오므라이스 1.10.15',
 '꼬치어묵우동',
 '돌솥비빔밥 1',
 '해신볶음밥 1.17',
 '뚝배기순두부찌개 5.

In [5]:
import re

weekday_tags = [tag for tag in textTags if re.match(r'\d{1,2}월 \d{1,2}일 \(\w\)', tag.text) is not None]

[t.text for t in weekday_tags]

['11월 11일 (월)',
 '11월 12일 (화)',
 '11월 13일 (수)',
 '11월 14일 (목)',
 '11월 15일 (금)',
 '11월 16일 (토)',
 '11월 17일 (일)']

In [6]:
lefts = []

for t in weekday_tags:
    style = t.get_attribute('style')
    left_ = re.search(r'left:\s*(\d+\.?\d*)px', style)
    if left_ is None:
        print('left', style)
        continue
    lefts.append(float(left_.group(1)))

lefts

[126.739, 199.629, 273.516, 346.688, 420.285, 493.661, 566.552]

In [7]:
from collections import defaultdict

def find_closest_left(left: float, lefts: list[float]) -> float:
    return min(lefts, key=lambda x: abs(x - left))

tags_by_left = defaultdict(list)

for tag in textTags:
    style = tag.get_attribute('style')
    if style is None:
        continue

    left_ = re.search(r'left:\s*(\d+\.?\d*)px', style)
    if left_ is None:
        continue
    left = float(left_.group(1))

    closest_left = find_closest_left(left, lefts)
    if abs(closest_left - left) > 30:
        continue

    tags_by_left[closest_left].append(tag)

strs_by_left = defaultdict(list)
# tags_by_left 에 넣을 때 top값이 비슷하면 string concat

for left, tags in tags_by_left.items():
    tags = sorted(tags, key=lambda x: float(re.search(r'top:\s*(\d+\.?\d*)px', x.get_attribute('style')).group(1)))

    last_top = None
    for tag in tags:
        top_ = re.search(r'top:\s*(\d+\.?\d*)px', tag.get_attribute('style'))
        if top_ is None:
            continue

        top = float(top_.group(1))
        if tag.text == "---":
            strs_by_left[left].extend(["---"] * 2)
        elif last_top is not None and abs(top - last_top) < 5:
            strs_by_left[left][-1] += tag.text
        else:
            strs_by_left[left].append(tag.text)

        last_top = top

strs_by_left

defaultdict(list,
            {126.739: ['11월 11일 (월)',
              '쌀밥 & 햄볶음밥 10.15',
              '어묵무국',
              '계란후라이 1',
              '깻잎닭갈비 15',
              '청경채오리엔탈무침',
              '배추김치',
              '야채샐러드',
              '시리얼&우유 2.6',
              '토스트&잼 6',
              '함박오므라이스 1.10.15',
              '마늘빵 6',
              '흑미밥',
              '어묵무국',
              '제육고추장볶음 10',
              '미역줄기볶음',
              '해물볶음우동 17.18',
              '콩나물무침',
              '배추김치',
              '흑미밥',
              '시래기된장국',
              '고등어김치찜 7',
              '생선가스',
              '민찌로제파스타 10',
              '타르소스',
              '깍두기',
              '1. 계란류egg 2. 우유milk 3. 메밀buckwheat 4. 땅콩peanut 5. 대두soybean 6. 밀heat 7. 고등어mackerel 8 .게crab 9. 새우shrimp 10. 돼지고기pork 11. 복숭아peach',
              '12. 토마토tomato 13. 아황산류sulgite 14. 호두walnut 15. 닭고기chicken 16. 쇠고기beef 17. 오징어squid 18. 조개류shellfish(굴, 전복, 홍합 등) 19. 잣pinenut'],
             199.629: ['11월 12일

In [8]:
meal_lists = [week_meal[1:] for week_meal in strs_by_left.values()]

def list_to_dict(meal_list: list[str]) -> dict[str, list[str]]:
    return {
        'breakfast': meal_list[:6],
        'simple_breakfast': meal_list[6:9],
        'rich_lunch': meal_list[9:11],
        'lunch': meal_list[11:18],
        'dinner': meal_list[18:25]
    }

meals = [list_to_dict(week_meal) for week_meal in meal_lists]

meals

[{'breakfast': ['쌀밥 & 햄볶음밥 10.15',
   '어묵무국',
   '계란후라이 1',
   '깻잎닭갈비 15',
   '청경채오리엔탈무침',
   '배추김치'],
  'simple_breakfast': ['야채샐러드', '시리얼&우유 2.6', '토스트&잼 6'],
  'rich_lunch': ['함박오므라이스 1.10.15', '마늘빵 6'],
  'lunch': ['흑미밥',
   '어묵무국',
   '제육고추장볶음 10',
   '미역줄기볶음',
   '해물볶음우동 17.18',
   '콩나물무침',
   '배추김치'],
  'dinner': ['흑미밥',
   '시래기된장국',
   '고등어김치찜 7',
   '생선가스',
   '민찌로제파스타 10',
   '타르소스',
   '깍두기']},
 {'breakfast': ['쌀밥 & 카레볶음밥 10.15',
   '소고기미역국 16',
   '계란후라이 1',
   '돈육메추리알장조림 1.10',
   '고구마순볶음',
   '배추김치'],
  'simple_breakfast': ['야채샐러드', '시리얼&요거트 2.6', '토스트&잼 6'],
  'rich_lunch': ['꼬치어묵우동', '후리가케주먹밥'],
  'lunch': ['흑미밥',
   '소고기미역국 16',
   '후르츠탕수육 10.15',
   '마파두부 5.10',
   '비빔만두 10',
   '짜사이볶음',
   '배추김치'],
  'dinner': ['흑미밥',
   '부대찌개 & 쫄면사리 10.15',
   '맛살야채계란찜 1',
   '명엽채볶음',
   '무생채',
   '깻잎장아찌',
   '배추김치']},
 {'breakfast': ['11월 13일 (수)',
   '쌀밥 & 굴소스볶음밥 10.15',
   '얼큰콩나물국',
   '계란후라이 1',
   '팝콘치킨 & 머스타드 15',
   '고추지무침'],
  'simple_breakfast': ['배추김치', '야채샐러드', '시리얼&우유 2.